In [2]:
import config
import mysql.connector 
import requests
import json
from bs4 import BeautifulSoup
from mysql.connector import errorcode
import time

cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'Shark'
)
cursor = cnx.cursor()

In [3]:
import pandas as pd
df_ny = pd.read_csv('NY_NEW_YORK_CITY_2019.csv')
df_al = pd.read_csv('NY_ALBANY_2019.csv')
df_roc = pd.read_csv('NY_ROCHESTER_2019.csv')
df_sr = pd.read_csv('NY_SYRACUSE_2019.csv')
df_yap = pd.read_csv('NY_YAPHANK_2019.csv')
cities = [df_ny, df_al, df_roc, df_sr, df_yap]

In [4]:
df_ny.head()

,LOCATION_NAME,SAMPLE COLLECTION TIME,DOSE EQUIVALENT RATE (nSv/h),GAMMA COUNT RATE R02 (CPM),GAMMA COUNT RATE R03 (CPM),GAMMA COUNT RATE R04 (CPM),GAMMA COUNT RATE R05 (CPM),GAMMA COUNT RATE R06 (CPM),GAMMA COUNT RATE R07 (CPM),GAMMA COUNT RATE R08 (CPM),GAMMA COUNT RATE R09 (CPM),STATUS
0,NY: NEW YORK CITY,01/28/2019 15:52:00,97.0,2375.0,1322.0,393.0,199.0,133.0,150.0,85.0,40.0,APPROVED
1,NY: NEW YORK CITY,01/28/2019 16:52:00,98.0,2377.0,1325.0,389.0,203.0,137.0,149.0,82.0,41.0,APPROVED
2,NY: NEW YORK CITY,01/28/2019 17:53:00,102.0,2374.0,1328.0,388.0,203.0,133.0,152.0,86.0,40.0,APPROVED
3,NY: NEW YORK CITY,01/28/2019 18:53:00,93.0,2366.0,1323.0,393.0,201.0,137.0,150.0,82.0,41.0,APPROVED
4,NY: NEW YORK CITY,01/28/2019 19:53:00,95.0,2355.0,1332.0,394.0,207.0,136.0,148.0,86.0,41.0,APPROVED


In [5]:
import numpy as np
def clean_df(city):
#     remove rest of mouth 
    city['SAMPLE COLLECTION TIME'] = [x[0:2] for x in city['SAMPLE COLLECTION TIME']]
#     remove ny in location
    city['LOCATION_NAME'] = [x[3:] for x in city['LOCATION_NAME']]
#     replace nan with main
    city['DOSE EQUIVALENT RATE (nSv/h)'] = city['DOSE EQUIVALENT RATE (nSv/h)'].replace(np.nan, city['DOSE EQUIVALENT RATE (nSv/h)'].mean())
    city['GAMMA COUNT RATE R02 (CPM)'] = city['GAMMA COUNT RATE R02 (CPM)'].replace(np.nan, city['GAMMA COUNT RATE R02 (CPM)'].mean())
    city['GAMMA COUNT RATE R03 (CPM)'] = city['GAMMA COUNT RATE R03 (CPM)'].replace(np.nan, city['GAMMA COUNT RATE R03 (CPM)'].mean())
    city['GAMMA COUNT RATE R04 (CPM)'] = city['GAMMA COUNT RATE R04 (CPM)'].replace(np.nan, city['GAMMA COUNT RATE R04 (CPM)'].mean())
    city['GAMMA COUNT RATE R05 (CPM)'] = city['GAMMA COUNT RATE R05 (CPM)'].replace(np.nan, city['GAMMA COUNT RATE R05 (CPM)'].mean())
    

In [11]:
df_yap.isna().sum()

LOCATION_NAME                    0
SAMPLE COLLECTION TIME           0
DOSE EQUIVALENT RATE (nSv/h)     0
GAMMA COUNT RATE R02 (CPM)       0
GAMMA COUNT RATE R03 (CPM)       0
GAMMA COUNT RATE R04 (CPM)       0
GAMMA COUNT RATE R05 (CPM)       0
GAMMA COUNT RATE R06 (CPM)      29
GAMMA COUNT RATE R07 (CPM)      29
GAMMA COUNT RATE R08 (CPM)      29
GAMMA COUNT RATE R09 (CPM)      29
STATUS                           0
dtype: int64

In [6]:
for city in cities:
    clean_df(city)

In [ ]:
# TABLES = {} #use dictionary to save diff create statements for each table
# TABLES['RADNYC'] = (
#     "  CREATE TABLE RADNYC ("
#     "  id int(11) NOT NULL AUTO_INCREMENT primary key,"
#     "  Location varchar(200) not null,"
#     "  Sample_Month int,"
#     "  Dose_Equivalent_Rate float,"
#     "  Gamma_Count_Rate_min_2 float,"
#     "  Gamma_Count_Rate_min_3 float,"
#     "  Gamma_Count_Rate_min_4 float,"
#     "  Gamma_Count_Rate_min_5 float,"
#     "  Status varchar(200)"
        
# ") ENGINE=InnoDB")



# for table_name in TABLES:
#     table_description = TABLES[table_name]
#     try:
#         print("Creating table {}: ".format(table_name), end='')
#         cursor.execute(table_description)
#     except mysql.connector.Error as err:
#         if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
#             print("already exists.")
#         else:
#             print(err.msg)
#     else:
#         print("OK")


In [12]:
# insert certain datain table
for city in cities:
    for row in range(len(city)):
        hold= city.loc[row]
        tuple_=(hold["LOCATION_NAME"], hold["SAMPLE COLLECTION TIME"], hold["DOSE EQUIVALENT RATE (nSv/h)"], hold["GAMMA COUNT RATE R02 (CPM)"],
               hold["GAMMA COUNT RATE R03 (CPM)"], hold["GAMMA COUNT RATE R04 (CPM)"],
               hold["GAMMA COUNT RATE R05 (CPM)"], hold["STATUS"])
        insert_stmt = "INSERT INTO RADNYC ( Location, Sample_Month, Dose_Equivalent_Rate, Gamma_Count_Rate_min_2,Gamma_Count_Rate_min_3, Gamma_Count_Rate_min_4, Gamma_Count_Rate_min_5, Status) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(insert_stmt, tuple_)
        cnx.commit()



In [ ]:
df.boxplot("GAMMA COUNT RATE R02 (CPM)", by= "SAMPLE COLLECTION TIME", figsize= (6,6))